In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine
from ling_features import tone_count

conn_string = 'postgresql://10.101.13.99/crsp'

def expand_json(df, col):
    return pd.concat([df.drop([col], axis=1),
                      df[col].map(lambda x: json.loads(json.dumps(x))).apply(pd.Series)], axis=1)

engine = create_engine(conn_string)

sql =  """
SELECT file_name, last_update, speaker_number, context, section,
   speaker_text
FROM streetevents.speaker_data
WHERE file_name = '1000000_T' 
ORDER BY section, context, speaker_number"""

speaker_data = pd.read_sql(sql, engine)

In [2]:
speaker_data['last_update'] = speaker_data['last_update'].map(lambda x: str(x.astimezone('UTC')))
speaker_data['tone_data'] = speaker_data['speaker_text'].apply(tone_count)
speaker_data = expand_json(speaker_data, 'tone_data')
speaker_data = speaker_data.drop(['speaker_text'], axis=1)

In [3]:
speaker_data

,file_name,last_update,speaker_number,context,section,positive,negative,uncertainty,litigious,modal_strong,modal_weak
0,1000000_T,2005-07-29 19:37:34+00:00,1,pres,1,0,2,1,0,2,1
1,1000000_T,2005-07-29 19:37:34+00:00,2,pres,1,2,4,3,1,6,2
2,1000000_T,2005-07-29 19:37:34+00:00,3,pres,1,26,11,9,1,7,1
3,1000000_T,2005-07-29 19:37:34+00:00,4,pres,1,11,7,8,2,5,4
4,1000000_T,2005-07-29 19:37:34+00:00,5,pres,1,26,3,6,1,8,3
...,...,...,...,...,...,...,...,...,...,...,...
78,1000000_T,2005-07-29 19:37:34+00:00,74,qa,1,0,0,0,0,0,0
79,1000000_T,2005-07-29 19:37:34+00:00,75,qa,1,0,0,0,0,0,0
80,1000000_T,2005-07-29 19:37:34+00:00,76,qa,1,0,1,0,0,0,0
81,1000000_T,2005-07-29 19:37:34+00:00,77,qa,1,0,1,1,0,0,1


In [4]:
sql =  """
SELECT *
FROM se_features.tone_measure
WHERE file_name = '1000000_T' 
ORDER BY section, context, speaker_number"""

orig_data = pd.read_sql(sql, engine)

In [5]:
orig_data

,file_name,last_update,speaker_number,context,section,positive,negative,uncertainty,litigious,modal_strong,modal_weak
0,1000000_T,2005-07-29 19:37:34+00:00,1,pres,1,0,2,1,0,2,1
1,1000000_T,2005-07-29 19:37:34+00:00,2,pres,1,0,4,3,1,6,2
2,1000000_T,2005-07-29 19:37:34+00:00,3,pres,1,24,10,9,1,6,1
3,1000000_T,2005-07-29 19:37:34+00:00,4,pres,1,9,7,7,2,5,3
4,1000000_T,2005-07-29 19:37:34+00:00,5,pres,1,26,3,5,1,8,2
...,...,...,...,...,...,...,...,...,...,...,...
78,1000000_T,2005-07-29 19:37:34+00:00,74,qa,1,0,0,0,0,0,0
79,1000000_T,2005-07-29 19:37:34+00:00,75,qa,1,0,0,0,0,0,0
80,1000000_T,2005-07-29 19:37:34+00:00,76,qa,1,0,1,0,0,0,0
81,1000000_T,2005-07-29 19:37:34+00:00,77,qa,1,0,1,1,0,0,1


In [6]:
# Please ignore the `False` in column `last_update`.
# It is `False` for the whole column because `last_update` in orig_data is in datetime format 
# While `last_update` in the new speaker_data is in str format
# However, they should be the same after uploading to the server with the dtype specification.

# Here please focus on the `False` in se_features columns such as `positive`.
speaker_data == orig_data

,file_name,last_update,speaker_number,context,section,positive,negative,uncertainty,litigious,modal_strong,modal_weak
0,True,False,True,True,True,True,True,True,True,True,True
1,True,False,True,True,True,False,True,True,True,True,True
2,True,False,True,True,True,False,False,True,True,False,True
3,True,False,True,True,True,False,True,False,True,True,False
4,True,False,True,True,True,True,True,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...
78,True,False,True,True,True,True,True,True,True,True,True
79,True,False,True,True,True,True,True,True,True,True,True
80,True,False,True,True,True,True,True,True,True,True,True
81,True,False,True,True,True,True,True,True,True,True,True
